In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats import chi2, norm, cauchy
from iminuit import Minuit
from iminuit.cost import LeastSquares
# Cambiare directory per spostarsi nella cartella principale
import os
while not os.getcwd().endswith('Laboratorio_SiPM'):
    os.chdir('..')

import sys
sys.path.append('Config')
from lablib import *

# Sodio 30 min

In [ ]:
df_bkg = read_corretto(path='Dati/Parte 2/LySO/Bkg_30min.txt', titles=['ADC', 'Counts'])
df = read_corretto(path = 'Dati/Parte 2/LySO/Sodio_30min.txt', titles=['ADC', 'Counts'])
df_nobkg = df[:] - df_bkg [:]
df_nobkg['ADC'] = df['ADC']
err_df_nobkg = np.array(np.sqrt(df['Counts'][:] + df_bkg['Counts'][:]))
# print(len(df), '\n', len(df_bkg), '\n', df_nobkg)
fig, ax = plt.subplots(figsize = (30, 6))
plt.title('Spettro di sodio meno background', fontsize = 20)
plt.grid(True, linewidth = 0.5)
plt.scatter(df_nobkg['ADC'], df_nobkg['Counts'], s = 0.8)
plt.errorbar(df['ADC'], df_nobkg['Counts'], yerr=  err_df_nobkg)
fig, ax = plt.subplots(figsize = (30, 6))
plt.errorbar(df['ADC'], df_bkg['Counts'], yerr=  np.array(np.sqrt(df_bkg['Counts'][:])), fmt='o', markersize=0.5)
plt.errorbar(df['ADC'], df['Counts'], yerr=  np.array(np.sqrt(df['Counts'][:])), fmt='o', markersize=0.5)
plt.legend(['Background', 'Sodium'])
plt.title('Spettro di sodio e background', fontsize = 20)

## primo picco

In [ ]:
start_index = 150 - 65
end_index = 210 - 65
df_primo_picco = df_nobkg.iloc[start_index:end_index]
plt.errorbar(df_primo_picco['ADC'], df_primo_picco['Counts'], yerr = err_df_nobkg[start_index:end_index])

In [ ]:
pars, vals, errs, *others = LS_fit(df_primo_picco['ADC'], df_primo_picco['Counts'], err_df_nobkg[start_index:end_index], gauss, mu = 10000, sigma = 1000, S = 2000)
for i in range(len(pars)):
    print(f'{pars[i]} = {vals[i]} +/- {errs[i]}')
S_1 = vals[1]
mu1 = vals[0]

# cobalto


In [ ]:
Co = read_corretto(path = 'Dati/Parte 2/LySO/Cobalto_30min.txt', titles=['ADC', 'Counts'])
Co_nobkg = Co[:] - df_bkg[:]
fig, axes = plt.subplots(2, 1,figsize = (30, 6))
axes[0].set_title('Spettro di cobalto + background', fontsize = 20)
axes[0].plot(Co['ADC'], Co['Counts'], label = 'Cobalto')
axes[1].plot(Co['ADC'], Co_nobkg['Counts'], label = 'Cobalto - Bkg')
axes[0].grid(True, linewidth = 0.5)
axes[1].grid(True, linewidth = 0.5)
#plt.plot(Co['ADC'], Co_nobkg['Counts'])
axes[0].scatter(df_bkg['ADC'], df_bkg['Counts'], s = 0.8)
# plt.errorbar(df['ADC'], df_nobkg['Counts'], yerr=  err_df_nobkg)
# fig, ax = plt.subplots(figsize = (30, 6))
# plt.errorbar(df['ADC'], df_bkg['Counts'], yerr=  np.array(np.sqrt(df_bkg['Counts'][:])), fmt='o', markersize=0.5)
# plt.errorbar(df['ADC'], df['Counts'], yerr=  np.array(np.sqrt(df['Counts'][:])), fmt='o', markersize=0.5)
# plt.legend(['Background', 'Sodium'])
# plt.title('Spettro di sodio e background', fontsize = 20)

In [ ]:
start_index = 94 - 65
end_index = 113 - 65
limit = [['mu', (1000, 5000)]]
pars, vals, errs, *others = LS_fit_lims(Co['ADC'][start_index:end_index], Co_nobkg['Counts'][start_index:end_index], 
                                        np.array(np.sqrt((Co[start_index:end_index]['Counts']) + df_bkg['Counts'][start_index:end_index])),
                                        gauss, mu = 2500, sigma = 600, S = 2000, limits = limit)
S_2 = vals[1]
mu2 = vals[0]

# Resolution

In [ ]:
risoluzione_Na = 2.355 *  S_1/ mu1
print('risoluzione sodio:', risoluzione_Na, '\n', 'risoluzione cobalto:', 2.355 * S_2 / mu2)